# Backtest: FX bar data

Tutorial for [NautilusTrader](https://nautilustrader.io/docs/) a high-performance algorithmic trading platform and event driven backtester.

[View source on GitHub](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/tutorials/backtest_fx_bars.ipynb).

:::info
We are currently working on this tutorial.
:::

## Overview

This tutorial runs through how to set up a `BacktestEngine` (low-level API) for a single 'one-shot' backtest run using FX bar data.

## Prerequisites

- Python 3.11+ installed
- [JupyterLab](https://jupyter.org/) or similar installed (`pip install -U jupyterlab`)
- [NautilusTrader](https://pypi.org/project/nautilus_trader/) latest release installed (`pip install -U nautilus_trader`)

## Imports

We'll start with all of our imports for the remainder of this tutorial.

In [1]:
from decimal import Decimal

from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.backtest.models import FillModel
from nautilus_trader.backtest.modules import FXRolloverInterestConfig
from nautilus_trader.backtest.modules import FXRolloverInterestModule
from nautilus_trader.config import LoggingConfig
from nautilus_trader.config import RiskEngineConfig
from nautilus_trader.examples.strategies.ema_cross import EMACross
from nautilus_trader.examples.strategies.ema_cross import EMACrossConfig
from nautilus_trader.model import BarType
from nautilus_trader.model import Money
from nautilus_trader.model import Venue
from nautilus_trader.model.currencies import JPY
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import TestDataProvider
from nautilus_trader.test_kit.providers import TestInstrumentProvider

## Set up backtest engine

In [2]:
# Initialize a backtest configuration
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
    logging=LoggingConfig(log_level="ERROR"),
    risk_engine=RiskEngineConfig(
        bypass=True,  # Example of bypassing pre-trade risk checks for backtests
    ),
)

# Build backtest engine
engine = BacktestEngine(config=config)

## Add simulation module

We can optionally plug in a module to simulate rollover interest. The data is available from pre-packaged test data.

In [3]:
provider = TestDataProvider()
interest_rate_data = provider.read_csv("short-term-interest.csv")
config = FXRolloverInterestConfig(interest_rate_data)
fx_rollover_interest = FXRolloverInterestModule(config=config)

Couldn't find test data directory, test data will be pulled from GitHub


## Add fill model

For this backtest we'll use a simple probabilistic fill model.

In [4]:
fill_model = FillModel(
    prob_fill_on_limit=0.2,
    prob_fill_on_stop=0.95,
    prob_slippage=0.5,
    random_seed=42,
)

## Add venue

For this backtest we just need a single trading venue which will be a similated FX ECN.

In [5]:
SIM = Venue("SIM")
engine.add_venue(
    venue=SIM,
    oms_type=OmsType.HEDGING,  # Venue will generate position IDs
    account_type=AccountType.MARGIN,
    base_currency=None,  # Multi-currency account
    starting_balances=[Money(1_000_000, USD), Money(10_000_000, JPY)],
    fill_model=fill_model,
    modules=[fx_rollover_interest],
)

## Add instruments and data

Now we can add instruments and data. For this backtest we'll pre-process bid and ask side bar data into quotes using a `QuoteTickDataWrangler`.

In [6]:
# Add instruments
USDJPY_SIM = TestInstrumentProvider.default_fx_ccy("USD/JPY", SIM)
engine.add_instrument(USDJPY_SIM)

# Add data
wrangler = QuoteTickDataWrangler(instrument=USDJPY_SIM)
ticks = wrangler.process_bar_data(
    bid_data=provider.read_csv_bars("fxcm/usdjpy-m1-bid-2013.csv"),
    ask_data=provider.read_csv_bars("fxcm/usdjpy-m1-ask-2013.csv"),
)
engine.add_data(ticks)

## Configure strategy

Next we'll configure and initialize a simple `EMACross` strategy we'll use for the backtest.

In [7]:
# Configure your strategy
config = EMACrossConfig(
    instrument_id=USDJPY_SIM.id,
    bar_type=BarType.from_str("USD/JPY.SIM-5-MINUTE-BID-INTERNAL"),
    fast_ema_period=10,
    slow_ema_period=20,
    trade_size=Decimal(1_000_000),
)

# Instantiate and add your strategy
strategy = EMACross(config=config)
engine.add_strategy(strategy=strategy)

## Run backtest

We now have everything required to run the backtest. Once the engine has completed running through all the data, a post-analysis report will be logged.

In [8]:
engine.run()

C:\Users\User\AppData\Local\Temp\ipykernel_5400\4112651484.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  engine.run()


## Generating reports

Additionally, we can produce various reports to further analyze the backtest result.

In [9]:
engine.trader.generate_account_report(SIM)

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2013-01-31 23:59:59.700000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,None,[],True,{}
2013-01-31 23:59:59.700000+00:00,9998164,0,9998164,JPY,SIM-001,MARGIN,None,[],True,{}
2013-02-01 01:35:00+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,None,[],False,{}
2013-02-01 01:35:00+00:00,9998164,0,9998164,JPY,SIM-001,MARGIN,None,[],False,{}
2013-02-01 07:55:00+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,None,"[{'type': 'MarginBalance', 'initial': '0', 'ma...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2013-02-28 21:10:00+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,None,[],False,{}
2013-02-28 21:10:00+00:00,8837942,0,8837942,JPY,SIM-001,MARGIN,None,[],False,{}
2013-03-01 00:00:00+00:00,8838043,278072,8559971,JPY,SIM-001,MARGIN,None,"[{'type': 'MarginBalance', 'initial': '0', 'ma...",True,{}
2013-03-01 00:00:00+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,None,"[{'type': 'MarginBalance', 'initial': '0', 'ma...",False,{}


In [10]:
engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20130201-013500-001-000-1,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-001,SIM-1-001,SIM-001,SIM-1-001,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,75b30ff4-2e24-4fa6-b0d2-2c8d0b039bb8,2013-02-01 01:35:00+00:00,2013-02-01 01:35:00+00:00
O-20130201-075500-001-000-2,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-002,SIM-1-001,SIM-001,SIM-1-002,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,f075a7dd-9bc1-4bdf-81cd-9bd01ffac424,2013-02-01 07:55:00+00:00,2013-02-01 07:55:00+00:00
O-20130201-075500-001-000-3,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-003,SIM-1-002,SIM-001,SIM-1-003,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,70f6c2e9-f840-42aa-ab13-ee4587a0c1d1,2013-02-01 07:55:00+00:00,2013-02-01 07:55:00+00:00
O-20130201-090500-001-000-4,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-004,SIM-1-002,SIM-001,SIM-1-004,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,3a1b4e67-5b7a-4c5c-a61b-44ade662c0c0,2013-02-01 09:05:00+00:00,2013-02-01 09:05:00+00:00
O-20130201-090500-001-000-5,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-005,SIM-1-003,SIM-001,SIM-1-005,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,140773e5-0308-4cd5-b8bb-f2ce6c2af58a,2013-02-01 09:05:00+00:00,2013-02-01 09:05:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20130228-173500-001-000-464,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-464,SIM-1-232,SIM-001,SIM-1-464,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,5452b150-b625-42ee-8b30-1107c1841534,2013-02-28 17:35:00+00:00,2013-02-28 17:35:00+00:00
O-20130228-173500-001-000-465,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-465,SIM-1-233,SIM-001,SIM-1-465,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,b583681f-e53b-4f0b-ae9e-ef144d84faf7,2013-02-28 17:35:00+00:00,2013-02-28 17:35:00+00:00
O-20130228-211000-001-000-466,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-1-466,SIM-1-233,SIM-001,SIM-1-466,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,587dc10d-99c9-4aab-a8dc-62e3db38cab9,2013-02-28 21:10:00+00:00,2013-02-28 21:10:00+00:00


In [11]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
SIM-1-001,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130201-013500-001-000-1,O-20130201-075500-001-000-2,BUY,FLAT,0,1000000,1359682500000000000,2013-02-01 01:35:00+00:00,1359705300000000000,2013-02-01 07:55:00+00:00,22800000000000,91.791,92.117,[3678 JPY],0.00355,322322 JPY
SIM-1-002,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130201-075500-001-000-3,O-20130201-090500-001-000-4,SELL,FLAT,0,1000000,1359705300000000000,2013-02-01 07:55:00+00:00,1359709500000000000,2013-02-01 09:05:00+00:00,4200000000000,92.117,92.193,[3686 JPY],-0.00083,-79686 JPY
SIM-1-003,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130201-090500-001-000-5,O-20130201-104000-001-000-6,BUY,FLAT,0,1000000,1359709500000000000,2013-02-01 09:05:00+00:00,1359715200000000000,2013-02-01 10:40:00+00:00,5700000000000,92.192,92.128,[3687 JPY],-0.00069,-67687 JPY
SIM-1-004,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130201-104000-001-000-7,O-20130201-125500-001-000-8,SELL,FLAT,0,1000000,1359715200000000000,2013-02-01 10:40:00+00:00,1359723300000000000,2013-02-01 12:55:00+00:00,8100000000000,92.128,92.146,[3686 JPY],-0.00020,-21686 JPY
SIM-1-005,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130201-125500-001-000-9,O-20130201-133500-001-000-10,BUY,FLAT,0,1000000,1359723300000000000,2013-02-01 12:55:00+00:00,1359725700000000000,2013-02-01 13:35:00+00:00,2400000000000,92.146,91.995,[3683 JPY],-0.00164,-154683 JPY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIM-1-230,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130228-155000-001-000-459,O-20130228-161500-001-000-460,SELL,FLAT,0,1000000,1362066600000000000,2013-02-28 15:50:00+00:00,1362068100000000000,2013-02-28 16:15:00+00:00,1500000000000,92.263,92.342,[3692 JPY],-0.00086,-82692 JPY
SIM-1-231,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130228-161500-001-000-461,O-20130228-173000-001-000-462,BUY,FLAT,0,1000000,1362068100000000000,2013-02-28 16:15:00+00:00,1362072600000000000,2013-02-28 17:30:00+00:00,4500000000000,92.342,92.305,[3693 JPY],-0.00040,-40693 JPY
SIM-1-232,BACKTESTER-001,EMACross-000,USD/JPY.SIM,SIM-001,O-20130228-173000-001-000-463,O-20130228-173500-001-000-464,SELL,FLAT,0,1000000,1362072600000000000,2013-02-28 17:30:00+00:00,1362072900000000000,2013-02-28 17:35:00+00:00,300000000000,92.305,92.345,[3693 JPY],-0.00043,-43693 JPY
